In [3]:
import os
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

messages=[
        {
            "role": "user",
            "content": "You are writing tests for my code",
        }
    ]

In [5]:
assistant = client.beta.assistants.create(
  name="Code Tester",
  instructions="You are a personal assistant to help me create tests for my code functions. Run my code and create tests.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4o",
)

In [6]:
thread = client.beta.threads.create()

In [7]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Please generate a test in python that tests my python function :def reverse_string(text): return text[::-1]"
)

In [8]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Alessa. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Certainly, Jane Doe! To test the `reverse_string` function, we can use Python's built-in `unittest` framework. Below is an example of how you can write a test case for the `reverse_string` function.

```python
import unittest

def reverse_string(text):
    return text[::-1]

class TestReverseString(unittest.TestCase):
    def test_reverse_string(self):
        # Test with a normal string
        self.assertEqual(reverse_string("hello"), "olleh")
        
        # Test with an empty string
        self.assertEqual(reverse_string(""), "")
        
        # Test with a single character
        self.assertEqual(reverse_string("a"), "a")
        
        # Test with a palindrome
        self.assertEqual(reverse_string("racecar"), "racecar")
        
        # Test with spaces
        self.assertEqual(reverse_string("hello world"), "dlrow olleh")
        
        # Test with numbers and special characters
        self.assertEqual(reverse_string("123 ABC!"), "!CBA 321")
       